In [1]:
import os
import sys
import numpy as np
import pandas as pd
from tqdm import tqdm
from copy import deepcopy

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [2]:
# path = '/home/vlad/HDD/GIT/CryptoLp DataScience/Projects/05. Imbalanced/BTCUSDT/BINANCE/2020_12/New Folder'
# files = sorted(os.listdir(path))
# main_df = pd.DataFrame()
# for file in files:     
#     file_df = pd.read_csv(f'{path}/{file}')
#     main_df = pd.concat([main_df, file_df], ignore_index=True)
#     print(f'{file}')
# main_df.to_csv('/home/vlad/HDD/GIT/CryptoLp DataScience/Projects/05. Imbalanced/BTCUSDT/BINANCE/2020_12/10days.csv', index=False)

In [3]:
orderbook_levels = 30
data_types = {}
for i in range(orderbook_levels):
    data_types[f'ask_{i}_price'] = np.float32
for i in range(orderbook_levels):
    data_types[f'bid_{i}_price'] = np.float32
for i in range(orderbook_levels):
    data_types[f'ask_{i}_size'] = np.float32
for i in range(orderbook_levels):
    data_types[f'bid_{i}_size'] = np.float32
data_types['datetime'] = np.object

In [4]:
main_df = pd.read_csv('/home/vlad/HDD/GIT/CryptoLp DataScience/Projects/05. Imbalanced/BTCUSDT/BINANCE/2020_12/10days.csv',
                      dtype=data_types)

In [5]:
main_df['datetime'] = np.array(main_df['datetime'].values, dtype=np.datetime64)
# main_df.index = range(len(main_df))

In [6]:
main_df.head()

,ask_0_price,ask_1_price,ask_2_price,ask_3_price,ask_4_price,ask_5_price,ask_6_price,ask_7_price,ask_8_price,ask_9_price,...,bid_21_size,bid_22_size,bid_23_size,bid_24_size,bid_25_size,bid_26_size,bid_27_size,bid_28_size,bid_29_size,datetime
0,19722.089844,19722.660156,19722.669922,19723.189453,19723.220703,19723.730469,19723.740234,19724.050781,19724.199219,19724.919922,...,0.060,0.360,1.707,0.100,0.770,0.232,3.506,0.300,2.550,2020-12-01 00:00:00.095
1,19722.089844,19722.660156,19722.669922,19723.189453,19723.220703,19723.730469,19723.740234,19724.050781,19724.199219,19724.919922,...,0.360,1.707,0.100,0.770,0.232,3.506,0.300,2.550,6.700,2020-12-01 00:00:00.315
2,19722.089844,19722.660156,19722.669922,19723.189453,19723.210938,19723.220703,19724.050781,19724.199219,19724.919922,19725.009766,...,0.600,0.096,0.060,0.360,1.707,0.100,0.770,0.232,3.506,2020-12-01 00:00:00.464
3,19720.089844,19720.720703,19720.740234,19722.070312,19722.080078,19722.089844,19723.220703,19723.599609,19724.919922,19725.009766,...,1.000,1.100,0.305,0.548,0.360,0.500,1.593,3.485,2.991,2020-12-01 00:00:00.603
4,19717.820312,19718.779297,19720.070312,19720.080078,19720.099609,19720.119141,19720.599609,19720.710938,19720.740234,19722.080078,...,0.006,1.000,0.360,0.500,3.485,2.991,1.200,1.593,1.747,2020-12-01 00:00:00.705


In [10]:
ask_price_lists = main_df[main_df.columns[:30]].values
# ask_sizes_list = main_df[main_df.columns[60:90]].iloc[0].values

In [18]:
main_df[main_df.columns[30:60]]

,bid_0_price,bid_1_price,bid_2_price,bid_3_price,bid_4_price,bid_5_price,bid_6_price,bid_7_price,bid_8_price,bid_9_price,...,bid_20_price,bid_21_price,bid_22_price,bid_23_price,bid_24_price,bid_25_price,bid_26_price,bid_27_price,bid_28_price,bid_29_price
0,19720.070312,19720.060547,19720.029297,19720.019531,19720.000000,19719.419922,19719.410156,19718.199219,19717.820312,19717.810547,...,19715.400391,19715.169922,19714.859375,19714.789062,19714.769531,19714.480469,19714.460938,19713.890625,19713.880859,19713.820312
1,19720.070312,19720.060547,19720.029297,19720.019531,19720.000000,19719.419922,19719.410156,19718.199219,19717.820312,19717.810547,...,19715.169922,19714.859375,19714.789062,19714.769531,19714.480469,19714.460938,19713.890625,19713.880859,19713.820312,19713.750000
2,19720.080078,19720.070312,19720.050781,19720.039062,19720.029297,19720.019531,19720.000000,19719.419922,19719.410156,19718.310547,...,19715.720703,19715.550781,19715.400391,19715.169922,19714.859375,19714.789062,19714.769531,19714.480469,19714.460938,19713.890625
3,19718.779297,19717.820312,19716.929688,19716.259766,19715.630859,19715.550781,19714.050781,19713.169922,19712.960938,19712.890625,...,19711.279297,19711.119141,19711.099609,19711.039062,19710.730469,19710.640625,19710.000000,19709.740234,19709.439453,19709.429688
4,19716.929688,19716.259766,19715.550781,19715.250000,19714.650391,19714.050781,19713.800781,19713.570312,19713.179688,19713.169922,...,19711.480469,19711.390625,19711.119141,19710.640625,19710.000000,19709.439453,19709.429688,19709.179688,19708.490234,19708.109375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8348772,18250.339844,18250.019531,18250.000000,18249.990234,18249.480469,18249.130859,18249.119141,18248.849609,18248.740234,18248.730469,...,18248.080078,18248.000000,18247.990234,18247.919922,18247.900391,18247.839844,18247.720703,18247.710938,18247.699219,18247.689453
8348773,18250.339844,18250.130859,18250.000000,18249.990234,18249.480469,18249.130859,18249.119141,18248.849609,18248.740234,18248.730469,...,18248.099609,18248.080078,18248.039062,18248.000000,18247.990234,18247.919922,18247.900391,18247.839844,18247.720703,18247.710938
8348774,18250.339844,18250.150391,18250.000000,18249.990234,18249.480469,18249.130859,18249.119141,18248.849609,18248.740234,18248.730469,...,18248.089844,18248.080078,18248.000000,18247.990234,18247.919922,18247.900391,18247.839844,18247.720703,18247.710938,18247.699219
8348775,18250.339844,18250.150391,18250.000000,18249.990234,18249.480469,18249.179688,18249.130859,18249.119141,18248.849609,18248.740234,...,18248.289062,18248.279297,18248.150391,18248.099609,18248.080078,18248.000000,18247.990234,18247.919922,18247.900391,18247.839844


In [9]:
from tqdm import tqdm

In [20]:
max_count = list()
index = 0
for ask_price_list in tqdm(ask_price_lists):
    #ask_sizes_list = main_df[main_df.columns[60:90]].iloc[j].values
    price = list()
    #volume = list()
    k = 0
    for i in range(0, len(ask_price_list) - 1):
        temp_price = ask_price_list[k: i + 1]    
        #temp_sizes = ask_sizes_list[k: i + 1]
        if temp_price[-1] >= temp_price[0]+0.5:
            price.append(temp_price[0])
            #volume.append(sum(temp_sizes[:-1]))
            k = i     

    temp_price = ask_price_list[k:]
    #temp_sizes = ask_sizes_list[k:]
    if (ask_price_list[-1] < temp_price[0] + 0.5):    
        price.append(temp_price[0])
        #volume.append(sum(temp_sizes))
    else:
        price.append(temp_price[0])
        #volume.append(sum(temp_sizes))
        price.append(ask_price_list[-1])
        #volume.append(ask_sizes_list[-1])
    max_count.append((index, len(price)))
    index += 1

100%|██████████| 8348777/8348777 [18:28<00:00, 7531.90it/s]


In [21]:
min(max_count)

(0, 10)

In [22]:
index

8348777

In [12]:

    

# i = 1
# if ask_price_list[i+1] >= ask_price_list[i]+0.5:
#     price.append( ask_price_list[i] )
#     volume.append( ask_sizes_list[i] )
# else:
#     price.append( ask_price_list[i] )
#     volume.append(ask_sizes_list[i]+ask_sizes_list[i+1])
#     ask_price_list.remove(ask_price_list[i+1])
#     ask_sizes_list[i] = volume[i]
#     ask_sizes_list.remove(ask_sizes_list[i+1])

In [14]:
len(price), len(volume)

(10, 10)